## FDW Crop Production Data Profiling - Somalia

In [2]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [3]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32639' # Somalia
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/SO_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#auth = tuple(json.loads(open('token.json', "r").read()))
exec(open('/Users/wanders7/Documents/Code/Logins/FDW.py').read())
auth = (username, password)
parameters = {
    'format': 'json',
    'country': 'Somalia',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1990	 | SO1990A1 | 18	| SO1990A2	| 74	| nan	| 0	|
----------------------------------------------- #


In [4]:
# Check code
df_origin[
     (df_origin['fnid'] == 'SO1990A21201') &
     (df_origin['season_year'] == 'Deyr 2019') &
     (df_origin['indicator'] == 'Quantity Produced') &
     (df_origin['product'] == 'Sorghum')
 ].T

,726
country,Somalia
source_organization,"FSNAU, Somalia"
source_document,"FSNAU, Somalia"
geographic_unit_full_name,"Hargeysa, Woqooyi Galbeed, Somalia"
geographic_unit_name,Hargeysa
fnid,SO1990A21201
admin_0,Somalia
admin_1,Woqooyi Galbeed
admin_2,Hargeysa
admin_3,None


In [4]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_2'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(',')[0])

# b) Delay one year of Deyr season data 
# This is from an ongoing discussion. Currently, we confirmed that the Deyr season data is year off. Actually, the source data (FSNAU) Deyr-season data is year-off.
# For example, a Deyr season data with a season date of 2019-10-01 (or a start date of 2020-02-01) corresponds to a start date of 2021-02-01 data (2020 OND season and 2021 Feb Harvest).
# 2022-11-09: FDW re-revised season_date and season_year as original form so now a season date of 2020-02-01 (or a start date of 2020-02-01) corresponds to a start date of 2021-02-01 data (2020 OND season and 2021 Feb Harvest).
idx_deyr = df['season_name'] == 'Deyr'
for col_date in ['start_date', 'period_date']:
    delayed = pd.to_datetime(df.loc[idx_deyr, col_date]) + pd.offsets.DateOffset(years=1)
    df.loc[idx_deyr, col_date] = delayed.dt.strftime('%Y-%m-%d')
# df.loc[idx_deyr,'season_year'] = df.loc[idx_deyr,'season_year'].map(lambda x: x[:5]+str(int(x[5:])+1))

# c) Control population groups
# Data of Awdal (SO1990A111) prior to 2017 has a population group (Borama (SO1990A21109) and Baki (SO1990A21102))
# This is not duplicated data, so we will link the data with an exception for one duplicated data point:
df = df[df['id'] != 3831320]
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Harvested')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([SO_Admin1_1990, SO_Admin2_1990], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 17,271
Removed 3,090 "Missing Value" points
4,159/4,318 "Area Harvested" points are retained.
1,850/4,317 "Area Planted" points are retained.
4,112/4,318 "Quantity Produced" points are retained.
4,060/4,318 "Yield" points are retained.
Current data points: 14,181

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1982 - 2022
3 grain types are found: Maize (Corn), Rice (Paddy), Sorghum
4 seasons are found: Gu (07-01), Deyr (02-01), Gu off-season (09-01), Deyr off-season (03-01)
3 crop production system are found: agro_pastoral, none, riverine
Data sources include:
[1] FSNAU, Somalia --- FSNAU, Somalia
Administrative-1 fnids: 14
Administrative-2 fnids: 47
2 reporting units are found: SO1990R22805, SO1990R22806

- Total production over time ------------------ #
season_name    Deyr Deyr off-season     Gu Gu off-season
                                      

- Somalia crop seasonal calendar </br>

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-somalia.png?itok=raaT51aR)

- FDW data consists of `SO1990A1` and `SO1990A2`.

| Year | Admin-1 | # units | Admin-2  | # units |
| :--- | :----:  | :----:  | :----:   | ---:    |
| 1990 | SO1990A1| 18      | **`SO1990A2`** | 74      |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_admin_shapes.png?raw=true)

<!-- - Most of reliable data are admin-2 level, and FDW has few data for admin-1 level. Therefore, we aggregate/merge admin-2 level data to represent admin-1 level. -->
- Afmadow (SO1990A22802) has two crop reporting units: Afmadow (SO1990R22805) and Afmadow (Hagar) (SO1990R22806). This is confirmed by FEWS NET crop reporting unit (./data/admin_boundaries/SO_Crop_Units_1990.shp). Since SO1990A22802 has only one off-season record, we aggregate SO1990R22805 and SO1990R22806 to replace SO1990A22802.

| District |Crop reporting units|
| :---:|:---:|
|Afmadow (SO1990A22802) | Afmadow (SO1990R22805) and  Hagar (SO1990R22806)|

- **`SO1990A2`** is used to represent the current admin-level 2 crop data.
- Somalia has four crop seasons: `Deyr`, `Gu`, `Gu off-season`, `Deyr off-season`.
- Somalia has three crop production systems: `agro_pastoral`, `riverine`, `None`.
- Somalia has no population group(s).

In Admin-2 level data, we found unrealistic records. So, we manually remove those records:
- 26 districts have the exactly same `1 mt/ha` yield values in `Deyr` season of `2001`.
- Three districts have unrealistic yield values in `Gu` season of `2005`: Bu'aale (11.65 mt/ha), Jilib (2.27 mt/ha), and Saakow (20.8 mt/ha).

In [5]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = SO_Admin2_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
df['season_name'] = df['season_name'].replace({'Gu off-season':'Gu-off', 'Deyr off-season':'Deyr-off'})
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [6]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(SO_Admin2_1990, SO_Admin2_1990, 'ADMIN2', 'ADMIN2', prod, epsg)
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, prod, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Manual Editing
link_ratio['SO1990A22802'][['SO1990R22805', 'SO1990R22806']] = 1.0
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

CBR is considered for 'SO1990A21604' as no record found in: ['SO1990A21604']
CBR is considered for 'SO1990A21202' as no record found in: ['SO1990A21202']
CBR is considered for 'SO1990A21102' as no record found in: ['SO1990A21102']
CBR is considered for 'SO1990A21703' as no record found in: ['SO1990A21703']
CBR is considered for 'SO1990A21802' as no record found in: ['SO1990A21802']
CBR is considered for 'SO1990A21103' as no record found in: ['SO1990A21103']
CBR is considered for 'SO1990A21603' as no record found in: ['SO1990A21603']
CBR is considered for 'SO1990A21605' as no record found in: ['SO1990A21605']
CBR is considered for 'SO1990A21104' as no record found in: ['SO1990A21104']
CBR is considered for 'SO1990A21601' as no record found in: ['SO1990A21601']
CBR is considered for 'SO1990A21503' as no record found in: ['SO1990A21503']
CBR is considered for 'SO1990A21502' as no record found in: ['SO1990A21502']
CBR is considered for 'SO1990A21606' as no record found in: ['SO1990A21606']

In [7]:
# Manual correction ----------------------------- #
# - Remove data before 1995
area_new = area_new.loc[1995:]
prod_new = prod_new.loc[1995:]
crop_new = prod_new/area_new
# - Unrealistic values in 2005 in Gu season
target = ['SO1990A22701','SO1990A22702','SO1990A22703']
# area_new.loc[2005,pd.IndexSlice[target,:,'Maize (Corn)','Gu',:]] = np.nan
# prod_new.loc[2005,pd.IndexSlice[target,:,'Maize (Corn)','Gu',:]] = np.nan
crop_new.loc[2005,pd.IndexSlice[target,:,'Maize (Corn)','Gu',:]] = np.nan
# - Districts have the same 1 mt/ha yield records in 2001 in Deyr season
temp = crop_new.loc[2001, pd.IndexSlice[:,:,'Maize (Corn)','Deyr']]
target = temp.index.get_level_values(0)[temp == 1]
# area_new.loc[2001,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
# prod_new.loc[2001,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
crop_new.loc[2001,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
# - Unrealistic values in 1995 in Dery season
target = ['SO1990A22605']
# area_new.loc[1995,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
# prod_new.loc[1995,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
crop_new.loc[1995,pd.IndexSlice[target,:,'Maize (Corn)','Deyr',:]] = np.nan
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of Planting and Harvest year and season
cs = {
    'Deyr': {'planting_month':{'02-01':'10-01'},'harvest_month':{'02-01':'03-01'}},
    'Gu': {'planting_month':{'07-01':'03-01'},'harvest_month':{'08-01':'08-01'}},
}
cy = {}
# cy = {'Deyr': {'planting_year':-1}}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

########################################################################
# Replace 2022 data with 2023 data (2022-11-10)
idx = (stack['season_name'] == 'Deyr') & (stack['harvest_year'] == 2023)
stack.loc[idx, ['planting_year','harvest_year']] -= 1
########################################################################

# Save data
fn_out = '../data/crop/adm_crop_production_SO.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_hdf('../data/crop/adm_crop_production_SO.hdf', stack)
save_hdf('../data/crop/adm_crop_production_SO_raw.hdf', df)
save_npz('../data/crop/adm_crop_production_SO_ratio.npz', link_ratio)

../data/crop/adm_crop_production_SO.csv is saved.
../data/crop/adm_crop_production_SO.hdf is saved.
../data/crop/adm_crop_production_SO_raw.hdf is saved.
../data/crop/adm_crop_production_SO_ratio.npz is saved.


## Visualization of production data

In [8]:
# Bar chart of national grain production
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [1995, df['year'].max()]
product_order = ['Sorghum','Maize','Rice']
for season_name in ['Gu','Deyr']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

../figures/SO_bar_natgrainprod_Gu.png is saved.
../figures/SO_bar_natgrainprod_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_bar_natgrainprod_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_bar_natgrainprod_Deyr.png?raw=true)

In [9]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Gu'],
    ['Maize','Deyr'],
    ['Sorghum','Gu'],
    ['Sorghum','Deyr'],    
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

../figures/SO_line_pay_Maize_Gu.png is saved.
../figures/SO_line_pay_Maize_Deyr.png is saved.
../figures/SO_line_pay_Sorghum_Gu.png is saved.
../figures/SO_line_pay_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Maize_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Maize_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_line_pay_Sorghum_Deyr.png?raw=true)

In [10]:
# Heatmap of seasonal data availability
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
code = {'Gu':1,'Deyr':10}
comb = {1:1,10:2,11:3,12:3,20:3,21:3,22:3}
comb_name = {1:'Gu',2:'Deyr',3:'Gu + Deyr'}
for product_name in ['Maize (Corn)','Sorghum']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = '../figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

../figures/SO_heat_seasondata_Maize (Corn).png is saved.
../figures/SO_heat_seasondata_Sorghum.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_Maize%20(Corn).png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_Sorghum.png?raw=true)

In [11]:
# Heatmap of crop production system(s) per crop type
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
]
code = {'none':1,'riverine':10,'agro_pastoral':100}
comb = {1:1,10:2,11:3,100:4,101:5,110:6,111:7}
comb_name = {1:'None',2:'Riverine',3:'Riverine + None',4:'Agro_pastoral',5:'Agro_pastoral + None',6:'Agro_pastoral + Riverine',7:'All'}
for product_name, season_name in product_season:
    data = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    footnote = 'Reported crop production system(s) in %s - %s - %s (uncorrected)' % ('Somalia', product_name, season_name)
    fn_save = '../figures/%s_heat_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotHeatCropSystem(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

../figures/SO_heat_cropsystem_Maize (Corn)_Gu.png is saved.
../figures/SO_heat_cropsystem_Maize (Corn)_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_heat_seasondata_SorghumSO_heat_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)

### Irrigated (or riverine) districts

In [12]:
from tools_graphic import PlotMapCropSystem
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
product_season = [
    ['Maize (Corn)', 'Gu'],
    ['Maize (Corn)', 'Deyr'],
    ['Sorghum', 'Gu'],
    ['Sorghum', 'Deyr']
]
for product_name, season_name in product_season:
    sub = df[(df['product'] == product_name)&(df['season_name'] == season_name)]
    footnote = 'Crop production system(s) in %s - %s - %s' % ('Somalia', product_name, season_name)
    fn_save = '../figures/%s_map_cropsystem_%s_%s.png' % (country_iso, product_name, season_name)
    fig = PlotMapCropSystem(sub, footnote, fn_save)
    # fig.show()

../figures/SO_map_cropsystem_Maize (Corn)_Gu.png is saved.
../figures/SO_map_cropsystem_Maize (Corn)_Deyr.png is saved.
../figures/SO_map_cropsystem_Sorghum_Gu.png is saved.
../figures/SO_map_cropsystem_Sorghum_Deyr.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Maize%20(Corn)_Deyr.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Sorghum_Gu.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/SO_map_cropsystem_Sorghum_Deyr.png?raw=true)

In [15]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'SO', 'Somalia'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('../data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Gu'],
    ['Sorghum', 'Gu'],
    ['Maize', 'Deyr'],
    ['Sorghum', 'Deyr']
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = '../figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)

../figures/crop_calibrated/SO_Maize_Gu_SO1990A21201.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A21203.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A21301.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A21303.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A21304.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22303.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22307.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22401.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22402.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22403.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22404.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22501.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22503.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22601.png is saved.
../figures/crop_calibrated/SO_Maize_Gu_SO1990A22602.png is saved.
../figures